# Import Libraries

In [1]:
from swarmintelligence import MemoryBasedGreyWolfOptimization, Utilization
import os
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import time # calculate CPU time
import pickle
import itertools

# import package metrics
from sewar.full_ref import *

# Load Data Testing

In [2]:
# load the model from disk
test_dataset_misc = pickle.load(open('datasets/test_dataset.pkl', 'rb'))
gray_misc_dataset = test_dataset_misc['gray']
rgb_misc_dataset = test_dataset_misc['rgb']
# gray_misc_dataset = dict(itertools.islice(gray_misc_dataset.items(), 2))

In [3]:
rgb_misc_dataset

{'AirplaneF16.tiff': array([[[181,   0,  98],
         [185, 206, 202],
         [165, 207, 197],
         ...,
         [150, 166, 206],
         [150, 164, 202],
         [131, 147, 196]],
 
        [[141,   0, 108],
         [199, 193, 204],
         [195, 189, 200],
         ...,
         [162, 182, 204],
         [141, 155, 195],
         [119, 139, 197]],
 
        [[141,   0, 108],
         [197, 196, 203],
         [193, 193, 199],
         ...,
         [174, 195, 209],
         [144, 159, 196],
         [116, 144, 193]],
 
        ...,
 
        [[160,   0, 172],
         [210, 215, 212],
         [211, 214, 214],
         ...,
         [181, 185, 191],
         [173, 175, 190],
         [158, 144, 168]],
 
        [[163,   0, 173],
         [210, 216, 213],
         [210, 215, 215],
         ...,
         [168, 163, 177],
         [184, 182, 190],
         [167, 164, 184]],
 
        [[ 35, 127,  32],
         [ 33, 123,  34],
         [ 34, 125,  38],
         ...,
        

# Load Optimal Parameter

In [4]:
optimal_params = pickle.load(open('results/hyperparameter_tuning/memGWO/optimal_params.pkl', 'rb'))
optimal_params

{'wolfSize': 25, 'maxIteration': 100, 'crossover_rate': 1.0}

# Evaluation

In [5]:
thresholds = [2,3,4,5]
fitness_functions = ['kapur_entropy']
objs = ['max']
n_runs = 30

In [6]:
gwo_evaluation_results = {
    'image_name': [],
    'thresholds': [],
    'fitness_function': [],
    'obj':[],
    'Mean best_thresholds R':[],
    'Mean best_thresholds G':[],
    'Mean best_thresholds B':[],
    'Mean Fitness R':[],
    'Mean Fitness G':[],
    'Mean Fitness B':[],
    'Mean CPU_time (seconds)': [],
    'Mean MSE':[],
    'Mean RMSE':[],
    'Mean PSNR':[],
    'Mean SSIM':[],
    'Mean UQI':[],
    'Regions R': [],
    'Regions G': [],
    'Regions B': [],
    'Fitness R':[],
    'Fitness G':[],
    'Fitness B':[],
    'CPU_time (seconds)': [],
    'MSE':[],
    'RMSE':[],
    'PSNR':[],
    'SSIM':[],
    'UQI':[],
}
for image_name, image_array in rgb_misc_dataset.items():
    # split citra to r,g,b component
    r,g,b = cv2.split(image_array)
    
    for threshold in thresholds:
        for fitness_function in fitness_functions:
            for obj in objs:
                gwo_evaluation_results['image_name'].append(image_name)
                gwo_evaluation_results['thresholds'].append(threshold)
                gwo_evaluation_results['fitness_function'].append(fitness_function)
                gwo_evaluation_results['obj'].append(obj)
                print('\t','='*30)
                print("\t",image_name, "| Threshold = ", threshold)
                print('\t','='*30)
                
                # untuk setiap algorithm run sebanyak n_runs lalu hitung rata-rata hasilnya
                list_cpu_time = []
                list_best_thresholds_r = []
                list_best_thresholds_g = []
                list_best_thresholds_b = []
                list_fitness_r = []
                list_fitness_g = []
                list_fitness_b = []
                list_mse = []
                list_rmse = []
                list_psnr = []
                list_ssim = []
                list_uqi = []
                for i_run in range(n_runs):
                    # PSO optimization
                    gwo_r = MemoryBasedGreyWolfOptimization(
                        k=threshold,
                        wolfSize=optimal_params['wolfSize'],
                        maxIteration=optimal_params['maxIteration'],
                        fitness_function=fitness_function,
                        crossover_rate=optimal_params['crossover_rate'],
                        obj='max'
                    )
                    gwo_g = MemoryBasedGreyWolfOptimization(
                        k=threshold,
                        wolfSize=optimal_params['wolfSize'],
                        maxIteration=optimal_params['maxIteration'],
                        fitness_function=fitness_function,
                        crossover_rate=optimal_params['crossover_rate'],
                        obj='max'
                    )
                    gwo_b = MemoryBasedGreyWolfOptimization(
                        k=threshold,
                        wolfSize=optimal_params['wolfSize'],
                        maxIteration=optimal_params['maxIteration'],
                        fitness_function=fitness_function,
                        crossover_rate=optimal_params['crossover_rate'],
                        obj='max'
                    )
                    # Evaluation Metrics
                    st = time.time()
                    greyWolfs_b, best_thresholds_b = gwo_r.fit_run(b)
                    greyWolfs_g, best_thresholds_g = gwo_g.fit_run(g)
                    greyWolfs_r, best_thresholds_r = gwo_b.fit_run(r)
                    et = time.time()
                    print(gwo_r.get_params_training_()['best_fitness_tracking'][-1], " | ", gwo_g.get_params_training_()['best_fitness_tracking'][-1]," | ", gwo_b.get_params_training_()['best_fitness_tracking'][-1])
                    regions_b = Utilization().digitize(b, best_thresholds_b)
                    regions_g = Utilization().digitize(g, best_thresholds_g)
                    regions_r = Utilization().digitize(r, best_thresholds_r)
                    merge_regions_RGB = cv2.merge((regions_r,regions_g,regions_b))
                    
                    list_cpu_time.append(et-st)
                    list_best_thresholds_r.append(best_thresholds_r)
                    list_best_thresholds_g.append(best_thresholds_g)
                    list_best_thresholds_b.append(best_thresholds_b)
                    list_fitness_r.append(gwo_r.get_params_training_()['best_fitness_tracking'][-1])
                    list_fitness_g.append(gwo_g.get_params_training_()['best_fitness_tracking'][-1])
                    list_fitness_b.append(gwo_b.get_params_training_()['best_fitness_tracking'][-1])
                    list_mse.append(mse(image_array, merge_regions_RGB))
                    list_rmse.append(rmse(image_array, merge_regions_RGB))
                    list_psnr.append(psnr(image_array, merge_regions_RGB))
                    list_ssim.append(ssim(image_array, merge_regions_RGB)[0])
                    list_uqi.append(uqi(image_array, merge_regions_RGB))
                
                mean_best_thresholds_r = np.round(np.array(list_best_thresholds_r).mean(axis=0)).astype('int')
                mean_best_thresholds_g = np.round(np.array(list_best_thresholds_g).mean(axis=0)).astype('int')
                mean_best_thresholds_b = np.round(np.array(list_best_thresholds_b).mean(axis=0)).astype('int')
                mean_regions_r=Utilization().digitize(r, mean_best_thresholds_r)
                mean_regions_g=Utilization().digitize(g, mean_best_thresholds_g)
                mean_regions_b=Utilization().digitize(b, mean_best_thresholds_b)
                gwo_evaluation_results['Mean best_thresholds R'].append(mean_best_thresholds_r)
                gwo_evaluation_results['Mean best_thresholds G'].append(mean_best_thresholds_g)
                gwo_evaluation_results['Mean best_thresholds B'].append(mean_best_thresholds_b)
                gwo_evaluation_results['Mean Fitness R'].append(np.mean(np.array(list_fitness_r)))
                gwo_evaluation_results['Mean Fitness G'].append(np.mean(np.array(list_fitness_g)))
                gwo_evaluation_results['Mean Fitness B'].append(np.mean(np.array(list_fitness_b)))
                gwo_evaluation_results['Mean CPU_time (seconds)'].append(np.mean(np.array(list_cpu_time)))
                gwo_evaluation_results['Mean MSE'].append(np.mean(np.array(list_mse)))
                gwo_evaluation_results['Mean RMSE'].append(np.mean(np.array(list_rmse)))
                gwo_evaluation_results['Mean PSNR'].append(np.mean(np.array(list_psnr)))
                gwo_evaluation_results['Mean SSIM'].append(np.mean(np.array(list_ssim)))
                gwo_evaluation_results['Mean UQI'].append(np.mean(np.array(list_uqi)))
                gwo_evaluation_results['Regions R'].append(mean_regions_r)
                gwo_evaluation_results['Regions G'].append(mean_regions_g)
                gwo_evaluation_results['Regions B'].append(mean_regions_b)
                gwo_evaluation_results['Fitness R'].append(list_fitness_r)
                gwo_evaluation_results['Fitness G'].append(list_fitness_g)
                gwo_evaluation_results['Fitness B'].append(list_fitness_b)
                gwo_evaluation_results['CPU_time (seconds)'].append(list_cpu_time)
                gwo_evaluation_results['MSE'].append(list_mse)
                gwo_evaluation_results['RMSE'].append(list_rmse)
                gwo_evaluation_results['PSNR'].append(list_psnr)
                gwo_evaluation_results['SSIM'].append(list_ssim)
                gwo_evaluation_results['UQI'].append(list_uqi)
                print(f"Mean Fitness = {np.mean([np.mean(list_fitness_r),np.mean(list_fitness_g),np.mean(list_fitness_b)])}")

	 AirplaneF16.tiff | Threshold =  2
11.516335925490615  |  12.52692490343442  |  12.14148096951028
11.516335925490615  |  12.527011119424301  |  12.14148096951028
11.516335925490615  |  12.527011119424301  |  12.14148096951028
11.516335925490615  |  12.527011119424301  |  12.14148096951028
11.516335925490615  |  12.527011119424301  |  12.14148096951028
11.516335925490615  |  12.527011119424301  |  12.14148096951028
11.516335925490615  |  12.527011119424301  |  12.14148096951028
11.516335925490615  |  12.527011119424301  |  12.14148096951028
11.516335925490615  |  12.527011119424301  |  12.14148096951028
11.516335925490615  |  12.527011119424301  |  12.14148096951028
11.516335925490615  |  12.527011119424301  |  12.14148096951028
11.516335925490615  |  12.527011119424301  |  12.14148096951028
11.516335925490615  |  12.527011119424301  |  12.14148096951028
11.516335925490615  |  12.527011119424301  |  12.14148096951028
11.516335925490615  |  12.527011119424301  |  12.14148096951028
11.51

Mean Fitness = 20.682667653722216
	 Lena.png | Threshold =  2
11.612067412579773  |  12.698959273000352  |  12.068814847692082
11.612067412579773  |  12.698959273000352  |  12.068814847692082
11.612067412579773  |  12.698959273000352  |  12.068814847692082
11.612067412579773  |  12.698959273000352  |  12.068814847692082
11.612067412579773  |  12.698959273000352  |  12.068814847692082
11.612067412579773  |  12.698959273000352  |  12.068814847692082
11.612067412579773  |  12.698959273000352  |  12.068814847692082
11.612067412579773  |  12.698959273000352  |  12.068814847692082
11.612067412579773  |  12.698959273000352  |  12.068814847692082
11.612067412579773  |  12.698959273000352  |  12.06876347918707
11.612067412579773  |  12.698959273000352  |  12.068814847692082
11.612067412579773  |  12.698959273000352  |  12.068814847692082
11.612067412579773  |  12.698959273000352  |  12.068814847692082
11.612067412579773  |  12.698959273000352  |  12.068814847692082
11.612067412579773  |  12.698

Mean Fitness = 20.292401980606332
	 Male.tiff | Threshold =  2
12.634985859695353  |  12.634985859695353  |  12.634985859695353
12.634985859695353  |  12.634985859695353  |  12.634985859695353
12.634985859695353  |  12.634985859695353  |  12.634985859695353
12.634985859695353  |  12.634985859695353  |  12.634985859695353
12.634985859695353  |  12.634985859695353  |  12.634985859695353
12.634985859695353  |  12.634985859695353  |  12.634985859695353
12.634985859695353  |  12.634985859695353  |  12.634985859695353
12.634985859695353  |  12.634985859695353  |  12.634985859695353
12.634985859695353  |  12.634985859695353  |  12.634985859695353
12.634985859695353  |  12.634792970346426  |  12.634985859695353
12.634985859695353  |  12.634985859695353  |  12.634985859695353
12.634864676960278  |  12.634985859695353  |  12.634985859695353
12.634985859695353  |  12.634985859695353  |  12.634985859695353
12.634985859695353  |  12.634985859695353  |  12.634985859695353
12.634985859695353  |  12.6

Mean Fitness = 21.53716898260369
	 Mandrill.tiff | Threshold =  2
12.953566949356176  |  12.374487945108996  |  12.795287763878669
12.953537165138442  |  12.374487945108996  |  12.795287763878669
12.953566949356176  |  12.374487945108996  |  12.795287763878669
12.953537165138442  |  12.374487945108996  |  12.795287763878669
12.953566949356176  |  12.374487945108996  |  12.795287763878669
12.953566949356176  |  12.374487945108996  |  12.795287763878669
12.953566949356176  |  12.374487945108996  |  12.795287763878669
12.953566949356176  |  12.374487945108996  |  12.795287763878669
12.953566949356176  |  12.374487945108996  |  12.795287763878669
12.953566949356176  |  12.374487945108996  |  12.795287763878669
12.953566949356176  |  12.374487945108996  |  12.795287763878669
12.953537165138442  |  12.374487945108996  |  12.795287763878669
12.953566949356176  |  12.374487945108996  |  12.795287763878669
12.953566949356176  |  12.374487945108996  |  12.795287763878669
12.953566949356176  |  1

Mean Fitness = 21.634701910327777
	 Peppers.tiff | Threshold =  2
12.473224752357389  |  12.631191680976166  |  12.515792844374673
12.473224752357389  |  12.631191680976166  |  12.515792844374673
12.473224752357389  |  12.631191680976166  |  12.515792844374673
12.473224752357389  |  12.631191680976166  |  12.515792844374673
12.473224752357389  |  12.631191680976166  |  12.515792844374673
12.473224752357389  |  12.631191680976166  |  12.515792844374673
12.473224752357389  |  12.631191680976166  |  12.515792844374673
12.473224752357389  |  12.631191680976166  |  12.515792844374673
12.473224752357389  |  12.631191680976166  |  12.515792844374673
12.473224752357389  |  12.631191680976166  |  12.515792844374673
12.473224752357389  |  12.631191680976166  |  12.515792844374673
12.473224752357389  |  12.631191680976166  |  12.51540733581598
12.473224752357389  |  12.631191680976166  |  12.51540733581598
12.473224752357389  |  12.631191680976166  |  12.515792844374673
12.473224752357389  |  12.

21.07138161041314  |  21.22075776115444  |  20.96674139204441
Mean Fitness = 21.076526065990276
	 Sailboat on lake.tiff | Threshold =  2
12.353654080299911  |  12.913278798658428  |  12.012520001966921
12.353654080299911  |  12.913278798658428  |  12.012520001966921
12.353654080299911  |  12.913278798658428  |  12.106891348552676
12.353654080299911  |  12.913278798658428  |  12.106891348552676
12.353654080299911  |  12.913278798658428  |  12.012520001966921
12.353654080299911  |  12.913278798658428  |  12.012520001966921
12.353654080299911  |  12.913278798658428  |  12.106891348552676
12.353654080299911  |  12.913278798658428  |  12.012520001966921
12.353654080299911  |  12.913278798658428  |  12.106891348552676
12.353654080299911  |  12.913278798658428  |  12.106891348552676
12.353654080299911  |  12.913278798658428  |  12.012520001966921
12.353654080299911  |  12.913278798658428  |  12.012520001966921
12.353654080299911  |  12.913278798658428  |  12.012520001966921
12.353654080299911

20.959065385647797  |  21.73784140787453  |  20.89628966301833
20.954277099119857  |  21.726447443827666  |  20.89405293830608
Mean Fitness = 21.198365840494684


## Visualize Results using DataFrame

In [7]:
gwo_evaluation_results_df =pd.DataFrame(gwo_evaluation_results)
gwo_evaluation_results_df

,image_name,thresholds,fitness_function,obj,Mean best_thresholds R,Mean best_thresholds G,Mean best_thresholds B,Mean Fitness R,Mean Fitness G,Mean Fitness B,...,Regions B,Fitness R,Fitness G,Fitness B,CPU_time (seconds),MSE,RMSE,PSNR,SSIM,UQI
0,AirplaneF16.tiff,2,kapur_entropy,max,"[77, 169]","[79, 176]","[131, 187]",11.516326,12.527005,12.141481,...,"[[131, 255, 255, 255, 255, 255, 255, 255, 255,...","[11.516335925490615, 11.516335925490615, 11.51...","[12.52692490343442, 12.527011119424301, 12.527...","[12.14148096951028, 12.14148096951028, 12.1414...","[7.166840076446533, 6.86362886428833, 11.38708...","[2519.208491007487, 2523.781707763672, 2523.78...","[50.191717354634186, 50.23725418216716, 50.237...","[14.118162494447725, 14.110285725832608, 14.11...","[0.7685608079396896, 0.7683261151563018, 0.768...","[0.9328489107063768, 0.9327005178460971, 0.932..."
1,AirplaneF16.tiff,3,kapur_entropy,max,"[73, 124, 176]","[67, 126, 184]","[70, 134, 187]",14.750411,15.807633,15.343704,...,"[[134, 255, 255, 255, 255, 255, 255, 255, 255,...","[14.750426649535349, 14.750426649535349, 14.75...","[15.806520148696954, 15.807958945983751, 15.80...","[15.343723829257907, 15.343723829257907, 15.34...","[7.222255229949951, 7.830782413482666, 6.80622...","[2078.0015970865884, 2079.443375905355, 2079.4...","[45.58510279780653, 45.60091420032448, 45.6009...","[14.954344838615611, 14.951332620225305, 14.95...","[0.7915110682844962, 0.7910386396385488, 0.791...","[0.9488065960210604, 0.9488382499997597, 0.948..."
2,AirplaneF16.tiff,4,kapur_entropy,max,"[70, 109, 146, 182]","[53, 93, 138, 184]","[70, 111, 146, 190]",17.487681,18.681581,18.109031,...,"[[111, 255, 255, 255, 255, 255, 255, 255, 255,...","[17.49020950463806, 17.489425117350397, 17.490...","[18.681503352966732, 18.681620549079568, 18.68...","[18.10922361200402, 18.10922361200402, 18.1092...","[8.275252103805542, 8.264235734939575, 8.06626...","[1888.0112800598145, 1889.626890818278, 1885.5...","[43.4512517663164, 43.46983886349567, 43.42275...","[15.37075776174346, 15.36704300215906, 15.3764...","[0.7987038400118629, 0.7985017679152913, 0.798...","[0.9566679751151961, 0.9566042831852979, 0.956..."
3,AirplaneF16.tiff,5,kapur_entropy,max,"[65, 94, 124, 155, 184]","[38, 75, 113, 150, 187]","[69, 98, 128, 158, 192]",20.072558,21.383271,20.592174,...,"[[128, 255, 255, 255, 255, 255, 255, 255, 255,...","[20.07347092060286, 20.075210474384782, 20.072...","[21.38515681713851, 21.38605888349969, 21.3793...","[20.597018114173654, 20.593378163456215, 20.59...","[10.387528657913208, 8.427037477493286, 8.0079...","[1771.205815633138, 1770.984681447347, 1768.25...","[42.0856960930093, 42.08306882164545, 42.05065...","[15.648113313546979, 15.648655561948656, 15.65...","[0.7987234764280694, 0.7986935636888052, 0.799...","[0.9608303588605952, 0.9607786040966554, 0.961..."
4,Lena.png,2,kapur_entropy,max,"[134, 192]","[80, 150]","[96, 142]",11.612067,12.698956,12.068813,...,"[[142, 142, 142, 142, 142, 142, 142, 142, 142,...","[11.612067412579773, 11.612067412579773, 11.61...","[12.698959273000352, 12.698959273000352, 12.69...","[12.068814847692082, 12.068814847692082, 12.06...","[8.587990760803223, 9.957761287689209, 8.95540...","[1846.134838104248, 1846.134838104248, 1846.13...","[42.96667124765715, 42.96667124765715, 42.9666...","[15.46816942993814, 15.46816942993814, 15.4681...","[0.6426126551076533, 0.6426126551076533, 0.642...","[0.89487516545066, 0.89487516545066, 0.8948751..."
5,Lena.png,3,kapur_entropy,max,"[114, 159, 201]","[60, 109, 160]","[93, 133, 171]",14.466942,15.765747,15.022591,...,"[[133, 133, 171, 133, 133, 133, 133, 133, 133,...","[14.466963587850607, 14.466963587850607, 14.46...","[15.76575964298272, 15.76575964298272, 15.7657...","[15.022621134788034, 15.022621134788034, 15.02...","[8.350707530975342, 8.150330781936646, 7.96775...","[1040.7299410502117, 1040.7299410502117, 1040....","[32.26034626364404, 32.26034626364404, 32.2603...","[17.957423117872676, 17.957423117872676, 17.95...

# Save results

In [8]:
pickle.dump(gwo_evaluation_results_df, open('results/evaluation/memGWO/memGWO_kapur_RGB_df.pkl', 'wb'))
pickle.dump(gwo_evaluation_results, open('results/evaluation/memGWO/memGWO_kapur_RGB_dict.pkl', 'wb'))